### Simple document classification using Doc2Vec

In [2]:
#Import all the dependencies
import gensim
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join

In [3]:
docLabels = []
docLabels = [f for f in listdir('/home/sudharsan/Prac/doc2vec') if 
 f.endswith('.txt')]
#create a list data that stores the content of all text files in order of their names in docLabels
data = []
for doc in docLabels:
      data.append(open( doc).read())

In [6]:
docLabels

['doc2.txt', 'doc1.txt', 'dataanalytics.txt', 'datascience.txt']

In [7]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words('english'))
#This function does all cleaning of data using two objects above
def nlp_clean(data):
   new_data = []
   for d in data:
      new_str = d.lower()
      dlist = tokenizer.tokenize(new_str)
      dlist = list(set(dlist).difference(stopword_set))
      new_data.append(dlist)
   return new_data

In [8]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,    
[self.labels_list[idx]])

In [9]:
data = nlp_clean(data)

In [10]:
#iterator returned over all documents
it = LabeledLineSentence(data, docLabels)

In [11]:
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)

In [12]:
for epoch in range(100):
 model.train(it,total_examples=120, epochs=50)
 model.alpha -= 0.002
 model.min_alpha = model.alpha
 
 model.train(it,total_examples=120, epochs=50)
#saving the created model
model.save('doc2vec.model')
print 'model saved'

model saved


In [13]:
#loading the model
d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[1]
print docvec

[ 0.51677901  0.31258044 -0.65208143  0.38211957 -0.3599042  -0.18258128
 -0.14837013  0.12151566 -1.05993772 -0.44988567  0.31823382  0.01076245
  1.34790671 -0.2969535  -0.34067938 -0.92943388  0.1199168  -1.06547785
  0.10513712  1.07346499  0.50644928  0.12733546  1.03785872 -0.3308382
 -0.12249345  0.6081841  -0.3029393  -0.11394179 -0.50903821  0.34106588
  0.77057219  0.43314776 -0.94055104 -0.08001107 -0.22514546 -1.27893746
  1.31172657  0.6123817  -0.45724756 -0.01732037  0.18734178 -0.88062489
 -1.07519853 -0.56802016 -0.09084667  0.2063639  -1.24354613 -1.31610143
  0.10989806 -0.30430138 -0.7379021   0.42618427 -0.39330876  0.07784735
 -0.47767255  0.27728522 -1.05520236 -0.10190082  0.05903171 -0.12078369
  0.63227069  0.34822017 -0.32487381  0.79358453  0.97610193 -0.19995734
 -0.85193741 -1.80406082  0.38684225  0.69084144  1.08070779  0.03698773
 -0.41941971  0.80394578  1.22347963 -0.01613422  0.82045329 -0.46655262
 -0.84752238  1.19900119  0.50947136  0.96124256 -0.

In [14]:

#printing the vector of the file using its name
docvec = d2v_model.docvecs['doc1.txt'] #if string tag used in training


In [16]:

#to get most similar document with similarity scores using document-index
similar_doc = d2v_model.docvecs.most_similar(3) 
print similar_doc

[('doc2.txt', 0.9900029897689819), ('dataanalytics.txt', 0.9897094368934631), ('doc1.txt', -0.1383509784936905)]


In [17]:

#to get most similar document with similarity scores using document- name
sims = d2v_model.docvecs.most_similar('dataanalytics.txt')
print sims

[('doc2.txt', 0.9974048733711243), ('datascience.txt', 0.9897094368934631), ('doc1.txt', -0.21821844577789307)]


In [20]:

#to get vector of document that are not present in corpus 
new_vector = d2v_model.infer_vector('java.txt')


In [21]:

sims = d2v_model.docvecs.most_similar([new_vector])

In [22]:
sims

[('datascience.txt', 0.01247432827949524),
 ('dataanalytics.txt', 0.007095709443092346),
 ('doc2.txt', 0.003996063955128193),
 ('doc1.txt', -0.019129103049635887)]